# ENSE 471 Project
Quinn Bast<br/>
200352973

Assignment 3 Extension:<br/>
Brick Breaker

### Changes/Additions:
- Transformed the game into brick breaker!
    - This transformation was quite simple and much of the code from assignment 3 could be re-used. I figured creating a brick-breaker style game would be a uniue implementation and provide a lot of opportunity to apply new features
- Added Graphics
    - Having to use the kivy 'color' property is quite plain and makes the game look bland. I added textures and images to certain elements of the game to add a more unique and custom feel.
- Animations
    - Animations make or break games! Adding animations I cycle through a group of images on the update cycle to create particle effects and other animation features which add polish to the game
- Particle Effects!
    - Additional of particle effects to the game further adds a finishing polish to make certain effects in the game more distinguishable to the user
- User Profile system
    - Allows users to log in and out of the game. Allows users to 'save' their current game and re-load a level they were trying to complete.
- High score system
    - High scorse give a competitive edge to the game, encouraging players to compete for the highest score!

In [1]:

# write your world class .kv file here.
dotKV = '''\
#:import FadeTransition kivy.uix.screenmanager.FadeTransition

<TitleScreen>
    name: 'Title'
    size: (850, 480)
    BoxLayout:
        orientation: 'vertical'
        BoxLayout:
            size_hint_y: 1/5
            orientation: 'horizontal'
            Label:
                size_hint_x: 1/3
            Label:
                text: "Block Breaker!"
            Label:
                size_hint_x: 1/3
        BoxLayout:
            size_hint_y: 3/5
            orientation: 'horizontal'
            Label:
                size_hint_x: 1/5
            BoxLayout:
                orientation: 'vertical'
                Label:
                    text: "Player 1"
                ColorWheel:
                    id: p1color
                    color: [1, 0, 0, 1]
                Label:
                    text: "Color"
                    canvas:
                        Color:
                            rgba: p1color.color
                        Rectangle:
                            pos: self.pos
                            size: self.size
            Label:
                size_hint_x: 1/5
        BoxLayout:
            size_hint_y: 1/5
            orientation: 'horizontal'
            Label:
                size_hint_y: 1/3
            Button:
                text: "Play Game"
                on_press: root.parent.playGame()
            Label:
                size_hint_y: 1/3
        
<GameScreen>
    name: 'GameScreen'
    
<GameOver>
    name: 'GameOver'
    FloatLayout:
        pos: (0,0)
        size: (850,480)
        BoxLayout:
            orientation: 'vertical'
            Label:
                id: winText
            Button:
                text: "Play Again!"
                on_press: root.parent.titleScreen()

<World>
    canvas:
        Color:
            rgba: 0.5, 0.5, 0.5, 1
        Rectangle:
            pos: (0, 0)
            size: (self.width, self.height)
            
<Paddle>
    canvas:
        Color:
            rgba: self.color
        Rectangle:
            pos: (self.x - self.width/2, self.y - self.height/2)
            size: (self.width, self.height)
        Ellipse:
            pos: (self.x - self.width/2 - self.height/2, self.y - self.height/2)
            size: (self.height, self.height)
        Ellipse:
            pos: (self.x + self.width/2 - self.height/2, self.y - self.height/2)
            size: (self.height, self.height)
            
<Brick>
    canvas:
        Color:
            hsv: self.color
        Rectangle:
            pos: (self.x - self.width/2, self.y - self.height/2)
            size: (self.width, self.height)
            source: 'Images/brick.png'
        Color:
            rgba: 0, 0, 0, 1
        Rectangle:
            pos: (self.x - self.width/2, self.y - self.height/2)
            size: (self.width, self.height)
            source: self.images[5- self.hitsToBreak]
            
<Ball>
    canvas:
        Color:
            rgba: self.color
        Ellipse:
            pos: (self.x - self.r, self.y - self.r)
            size: (self.r * 2, self.r * 2)
            
<Animation>
    canvas:
        Color:
            rgba: [1, 1, 1, 1]
        Rectangle:
            pos: (self.x - self.width/2, self.y - self.height/2)
            size: (self.width, self.height)
            source: self.images[self.currentImage]
            
<Particle>
    canvas:
        Color:
            rgba: [1, 1, 1, 1]
        Rectangle:
            pos: (self.x - self.width/2, self.y - self.height/2)
            size: (self.width, self.height)
            source: self.source
            
\
'''
from kivy.lang import Builder
Builder.load_string(dotKV)

from kivy.config import Config
Config.set('graphics', 'fullscreen', '0')
Config.set('graphics', 'width', '850')
Config.set('graphics', 'height', '480')

from kivy.app import App
from kivy.uix.widget import Widget
from kivy.uix.label import Label
from kivy.uix.floatlayout import FloatLayout
from kivy.graphics import Rectangle, Color
from kivy.clock import Clock
from kivy.core.window import Window
from kivy.properties import ListProperty, NumericProperty, DictProperty, StringProperty
from kivy.uix.screenmanager import ScreenManager, Screen, FadeTransition
from random import random
import math
        
########################################################
########################################################
########################################################
##                                                    ##
##                     Game                           ##
##                                                    ##
########################################################
########################################################
########################################################
    
class Game(FloatLayout):
    def __init__(self, **kwargs):
        FloatLayout.__init__(self, **kwargs)
        self.world = World()
        Clock.schedule_interval(self.oneFrame, 1/60)
        self.add_widget(self.world)
        
    def oneFrame(self, args):
        return self.world.update(self)
    
########################################################
########################################################
########################################################
##                                                    ##
##                  Input Handler                     ##
##                                                    ##
########################################################
########################################################
########################################################            

class InputHandler(Widget):
    
    #Keys that the input handler should detect
    left = False
    right = False
    space = False
    
    def __init__ (self, **kwargs):
        Widget.__init__(self)
        self._keyboard = Window.request_keyboard(self._keyboard_closed, self)
        self._keyboard.bind(on_key_down=self._on_keyboard_down)
        self._keyboard.bind(on_key_up=self._on_keyboard_up)               
        
    def _keyboard_closed(self):
        self._keyboard.unbind(on_key_down=self._on_keyboard_down)
        self._keyboard = None

    def _on_keyboard_down(self, keyboard, keycode, text, modifiers):
        if keycode[1] == 'left':
            self.left = True
        if keycode[1] == 'right':
            self.right = True
        if keycode[1] == 'spacebar':
            self.space = True

    def _on_keyboard_up(self, keyboard, keycode):
        if keycode[1] == 'left':
            self.left = False
        if keycode[1] == 'right':
            self.right = False  
        if keycode[1] == 'spacebar':
            self.space = False
    
########################################################
########################################################
########################################################
##                                                    ##
##                     WORLD                          ##
##                                                    ##
########################################################
########################################################
########################################################
            
class World (Widget):
       
    def __init__(self, **kwargs):
        Widget.__init__(self, **kwargs)
        
        #Create the inputHandler
        self.inputHandler = InputHandler()
        
        #Create a list of collidable objects to check the balls collisions with
        self.collidables = []
        #Create a list of balls to check for collisions against
        self.balls = []
        #Create a list of animations
        self.animations = []
        
        #Randomly generate a level with a specific number of bricks.
        self.spawnBricks(30);
            
        #Add the player's paddle to the collidable list
        self.paddle = Paddle()
        self.collidables += [self.paddle]
        #Add the first ball to the list of balls.
        self.balls += [Ball()]
        
        #Add objects to the widget to render.
        for c in self.collidables:
            self.add_widget(c)
            
        for b in self.balls:
            self.add_widget(b)
    
    def update(self, *args):     
        #Move the paddle
        self.paddle.move()
        
        #move the ball(s) each update step
        for b in self.balls:
            b.move()
            
        #check for collisions after each of the balls have moved
        for b in self.balls:
            for c in self.collidables:
                b.collisionResolution(c)
                
        #Update any animations
        for a in self.animations:
            a.update()
                    
    #Generic spawn brick function which generates a level containing the specified number of bricks.
    def spawnBricks(self, noBricks):
        for i in range(0, noBricks):
            from random import randint
            locx = randint(20, 810);
            locy = randint(100, 440);
            noHits = randint(1, 5);
            color_h = randint(0, 100) / 100;
            color_s = randint(50, 100) / 100;

            brick = Brick(x = locx, y = locy, hitsToBreak = noHits, color = [color_h, color_s, 1.0])
            self.collidables += [brick]
 
    
########################################################
########################################################
########################################################
##                                                    ##
##                     COLLIDABLES                    ##
##                                                    ##
########################################################
########################################################
########################################################

#Basic collidable class. Anything that a ball can collide with will enherit from this class
class Collidable(Widget):
    x = NumericProperty(0) #X position
    y = NumericProperty(0) #Y position
    hitsToBreak = NumericProperty(1) #Number of hits required for the object to break
    color = ListProperty([1, 0, 0, 1]) #Color of the object.
    
    def __init__(self, **kwargs):
        Widget.__init__(self)
        self.color = kwargs.pop('color', [1, 0, 0, 1])
        self.x = kwargs.pop('x', 0)
        self.y = kwargs.pop('y', 0)
        self.hitsToBreak = kwargs.pop('hitsToBreak', 1)
        
    #Functino to remove objects from the game if they are broken
    def on_hitsToBreak(self, *args):
        #Don't remove the player's paddle
        if(not isinstance(self, Paddle)):
            if(self.hitsToBreak <= 0):
                self.parent.collidables.remove(self)
                explode = ExplosionEffect(x = self.x, y=self.y)
                self.parent.animations += [explode]
                self.parent.add_widget(explode)
                #Once this object is removed, it can't do anything. Do everything before its removed.
                self.parent.remove_widget(self)
        
#Class for any rectangular collidable object
class RectangleCollidable(Collidable):
    width = NumericProperty(30) #Pre-defined width and height
    height = NumericProperty(20)
    
    def __init__(self, **kwargs):
        Collidable.__init__(self, **kwargs)
        self.width = kwargs.pop('width', 30)
        self.height = kwargs.pop('height', 20)
        
    #Collision functinon. 
    #Because collisions occur different between different shapes
    def collide(self, ball, collisionPoint):
        #Function is passed a 'collisionPoint' which is the coordinate of the point where the collision is occurring.
        
        #Determine where the collision point is and what type of rebound should occur
        distanceToTop = collisionPoint[1] - self.y + self.height/2
        distanceToBot = collisionPoint[1] - self.y - self.height/2
        distanceToLef = collisionPoint[0] - self.x - self.width/2
        distanceToRig = collisionPoint[0] - self.x + self.width/2
        l = [distanceToTop, distanceToBot, distanceToLef, distanceToRig]
        
        closest = distanceToTop
        num = 0
        #Determine which edge of the rectangle is closest to the collision point.
        for i in range (0, 4):
            if(abs(l[i]) < closest):
                num = i
                closest = abs(l[i])
        
        if num == 0:
            #Hit the top, set the ball outside the object
            ball.y = self.y - self.height/2 - ball.r - 1
            ball.v_y = ball.v_y  * -1
        elif num == 1:
            #ball hit the bottom of the collidable
            ball.y = self.y + self.height/2 + ball.r + 1
            ball.v_y = ball.v_y  * -1
        elif num == 2:
            #ball hit the right side of the collidable
            ball.x = self.x + self.width/2 + ball.r + 1
            ball.v_x = ball.v_x  * -1
        elif num == 3:
            #ball hit the left side of the collidable
            ball.x = self.x - self.width/2 - ball.r -1
            ball.v_x = ball.v_x  * -1
        
        self.hitsToBreak -= 1

class SphericalCollidable(Collidable):
    r = NumericProperty(0)
    
    def __init__(self, **kwargs):
        Collidable.__init__(self)
        self.r = kwargs.pop('r', 0)
        
    #Collisions on spheres are much easier...
    def collide(self, ball):
        self.hitsToBreak -= 1
        ball.v_x = ball.v_x  * -1
        ball.v_y = ball.v_y  * -1
        
class Brick(RectangleCollidable):
    #Images which overlay the brick to show how close the brick is to being broken.
    images = ['Images/break1.png','Images/break2.png','Images/break3.png','Images/break4.png','Images/break5.png','Images/break5.png']
    
    def __init__(self, **kwargs):
        RectangleCollidable.__init__(self, **kwargs)
        
class Ball(Widget):
    
    #Properties
    x = NumericProperty(240)
    y = NumericProperty(25)
    r = NumericProperty(10)
    v_x = NumericProperty(0)
    v_y = NumericProperty(0)
    color = ListProperty([0, 1, 0, 1])
    
    def __init__(self, **kwargs):
        Widget.__init__(self, **kwargs)
        self.color = kwargs.pop('color', [0, 1, 0, 1])
        self.x = kwargs.pop('x', 240)
        self.y = kwargs.pop('y', 25)
        self.v_x = kwargs.pop('v_x', 0)
        self.v_y = kwargs.pop('v_y', 0)
        self.r = kwargs.pop('r', 10)
        
    def move(self):
        #If the spacebar is pressed and the ball is not moving, launch the ball to the right.
        #Ball will only ever not be moving at the beginning of a level
        if(self.parent.inputHandler.space and self.v_x == 0 and self.v_y == 0):
            self.v_x = 2
            self.v_y = 2
            
        #If the ball is not moving, move the ball along with the paddle before the level starts.
        #Allows the player to launch the ball from a specific location.
        if(self.v_x == 0 and self.v_y == 0):
                self.x = self.parent.paddle.x
        else:
            #If the ball is moving, keep it within the bounds of the screen and bounce off each edge.
            if(self.x + self.r > 850):
                self.v_x = self.v_x * -1
            if(self.x - self.r < 0):
                self.v_x = self.v_x * -1
            if(self.y + self.r > 480):
                self.v_y = self.v_y * -1
            if(self.y - self.r < 0):
                self.v_y = self.v_y * -1
                
            #Update the ball's position
            self.x += self.v_x
            self.y += self.v_y
    
    #Determine if the ball is colliding with anything.
    def collisionResolution(self, collidable):
        #If the ball is not moving (before a level starts), it should not detect collisions.
        if(self.v_x == 0 or self.v_y == 0):
            return
        
        #Determine what type the collidable object is being passed; spherical or rectangular.
        #Need to distinguish as spherical collision detection is different than rectangular.
        if(isinstance(collidable, SphericalCollidable)):
            #If the object is a sphere,
            #Check spherical collision condition
            if(pow(self.x - collidable.x, 2) + pow(self.y - collidable.y, 2) > (self.r + collidable.r)):
                return;
            
            #Handle collision if a collision occurred with a spherical object.
            collidable.collide(self)
            
        if(isinstance(collidable, RectangleCollidable)):
            #Holy. This took me a whole day to get working.
            
            #Create a list of 100 points which define points around the outer edge of the ball.
            #In order to do this I am using a unit circle (cos(t),sin(t))
            #Multiplying by the ball's radius and offestting by the ball's position.
            unitCircle = [((self.r * math.cos(2*math.pi/(n/50))) + self.x, (self.r * math.sin(2*math.pi/(n/50)) + self.y)) for n in range (1, 50)]
            
            #Find the point on the edge of the ball which is closest to the rectangle's center
            #This point determines if a collision is occurring.
            closestPoint = unitCircle[0]
            shortestDistance = math.sqrt(pow(unitCircle[0][0] - collidable.x, 2) + pow(unitCircle[0][0] - collidable.y, 2))
            for point in unitCircle:
                distance = math.sqrt(pow(point[0] - collidable.x, 2) + pow(point[1] - collidable.y, 2))
                if(distance < shortestDistance):
                    shortestDistance = distance
                    closestPoint = point

            #Get the rectangle edges
            top = collidable.y + collidable.height/2
            bottom = collidable.y - collidable.height/2
            left = collidable.x - collidable.width/2
            right = collidable.x + (collidable.width/2)
                    
            #Determine if the closest point is inside the box.
            if((closestPoint[0] > left and closestPoint[0] < right) #horizontally inside
               and (closestPoint[1] > bottom and closestPoint[1] < top)): #vertically inside)
               
                #If the point is inside the rectangle, perform rectangular collision
                collidable.collide(self, closestPoint)
        
########################################################
########################################################
########################################################
##                                                    ##
##                     Paddle                         ##
##                                                    ##
########################################################
########################################################
########################################################        
    
class Paddle(RectangleCollidable):
    
    #Properties
    color = ListProperty([1, 0, 0, 1])
        
    def __init__(self, **kwargs):
        Widget.__init__(self)
        self.pos = (200, 10)
        self.color = kwargs.pop('color', [1, 0, 0, 1])
        self.x = kwargs.pop('x', 240)
        self.y = kwargs.pop('y', 10)
        self.width = kwargs.pop('width', 50)
        self.height = kwargs.pop('height', 10)
        
    def move(self):
        #Move the paddle using the left and right arrow keys.
        #Moving the paddle directly changes the location.
        if (self.parent.inputHandler.left):
            self.x -= 5
        elif (self.parent.inputHandler.right):
            self.x += 5
        #Don't move outside the screen
        if(self.x - self.width/2 - self.height/2 < 0):
            self.x = self.width/2 + self.height/2
        elif(self.x + self.width/2 + self.height/2 > 850):
            self.x = 850 - (self.width/2 + self.height/2)
            
    def collide(self, ball, collisionPoint):
        #If the ball collides with the paddle, the location at which the ball hit the paddle should determine its velocity
        #Determine the total velocity of the ball
        totalVelocity = math.sqrt(pow(ball.v_x, 2) + pow(ball.v_y, 2))
        #This is the hypotenuse of the velocity triangle. This will not change in length.
        
        #Determine the ratio along the paddle where the collision point is:
        #This is the % from the left side of the paddle.
        ratio = (collisionPoint[0] - (self.x - self.width/2)) / self.width
        
        #This get the angle from the ratio from pi to 0
        angle = ((ratio - 1) * math.pi)
        
        #Get the expected value of the x velocity given y velocity of 1.5 and the angle determined above.
        ball.v_x = math.cos(angle) * 4
        #ball.v_y = totalVelocity * (1 - ratio)
        ball.v_y = ball.v_y * -1
        
        ball.y = self.y + self.height/2 + ball.r/2 + 2
        
########################################################
########################################################
########################################################
##                                                    ##
##                     ANIMATIONS                     ##
##                                                    ##
########################################################
########################################################
########################################################
               
#Basic animation class to run through animations
class Animation(Widget):
    
    images = ListProperty([]) #A list of the images in the animation to loop through
    duration = NumericProperty(0) #The duration of the animation
    currentFrame = 0
    currentImage = 0
    x = NumericProperty(0) #position to draw the animation
    y = NumericProperty(0)
    width = NumericProperty(0) #Width and height of the animation
    height = NumericProperty(0)
               
    def __init__(self, **kwargs):
        Widget.__init__(self)
        self.images = kwargs.pop('images', [])
        self.duration = kwargs.pop('duration', 1)
        self.x = kwargs.pop('x', 0)
        self.y = kwargs.pop('y', 0)
        self.width = kwargs.pop('width', 16)
        self.height = kwargs.pop('height', 16)
               
    def update(self):
        #determine the number of frames required to reach the maximum lifespan
        totalFrames = duration * 60
        #determine the number of frames per image.
        framesPerImage = totalFrames / len(images)
        currentFrame += 1
        currentImage = int(round(currentFrame / framesPerImage))
        
########################################################
########################################################
########################################################
##                                                    ##
##               Particle Effects                     ##
##                                                    ##
########################################################
########################################################
########################################################
               
#A single particle of a particle effect
class Particle(Widget):
    source = StringProperty("") #Image of the particle
    duration = NumericProperty(0) #Duration of the particle
    currentFrame = 0
    x = NumericProperty(0) #Position of the particle
    y = NumericProperty(0)
    v_x = NumericProperty(0) #velocity of the particle
    v_y = NumericProperty(0)
    width = NumericProperty(0) #size of the particle
    height = NumericProperty(0)
               
    def __init__(self, **kwargs):
        Widget.__init__(self)
        self.source = kwargs.pop('source', "")
        self.duration = kwargs.pop('duration', 1)
        self.x = kwargs.pop('x', 0)
        self.y = kwargs.pop('y', 0)
        self.v_x = kwargs.pop('v_x', 0)
        self.v_y = kwargs.pop('v_y', 0)
        self.width = kwargs.pop('width', 16)
        self.height = kwargs.pop('height', 16)
               
    def update(self):
        self.currentFrame += 1
        #determine the number of frames required to reach the maximum lifespan
        totalFrames = self.duration * 60
        #Scale the image size to the current progress of the animation
        scale = 1 - (self.currentFrame / totalFrames)
        self.width = self.width * scale
        self.height = self.height * scale
        self.x += self.v_x
        self.y += self.v_y
               
class ParticleEffect(Widget):
               
    particles = ListProperty([])
               
    def __init__(self, **kwargs):
        Widget.__init__(self)
               
    def update(self):
        for p in self.particles:
            p.update()
            if(p.currentFrame >= p.duration * 60):
                self.particles.remove(p)
        if len(self.particles) == 0:
            self.parent.animations.remove(self)
            self.parent.remove_widget(self)
               
class ExplosionEffect(ParticleEffect):
    def __init__(self, **kwargs):
        ParticleEffect.__init__(self, **kwargs)
        spawnx = kwargs.get('x', 0)
        spawny = kwargs.get('y', 0)
        from random import randint
        for i in range (0, 10):
            randx = randint(-30, 30)
            randy = randint(-30, 30)
            self.particles = self.particles + [Particle(source = "Particles/White.png", x = (spawnx + randx), y = (spawny + randy), v_x = randx, v_y = randy, duration = 1, width = 16, height = 16)]
        for p in self.particles:
            self.add_widget(p)
            
class PoofEffect(ParticleEffect):
    def __init__(self, **kwargs):
        ParticleEffect.__init__(self, **kwargs)
        spawnx = kwargs.get('x', 0)
        spawny = kwargs.get('y', 0)
        from random import randint
        for i in range (0, 10):
            randx = randint(-30, 30)
            randy = randint(-30, 30)
            randDuration = randint(10, 50) / 10
            self.particles = self.particles + [Particle(source = "Particles/White.png", x = (spawnx + randx), y = (spawny + randy), duration = randDuration, width = 16, height = 16)]
        for p in self.particles:
            self.add_widget(p)
            
class FallingEffect(ParticleEffect):
    def __init__(self, **kwargs):
        ParticleEffect.__init__(self, **kwargs)
        spawnx = kwargs.get('x', 0)
        spawny = kwargs.get('y', 0)
        from random import randint
        for i in range (0, 5):
            randx = randint(-15, 15)
            vy = randint(-5, 0)
            randDuration = randint(10, 50) / 10
            self.particles = self.particles + [Particle(source = "Particles/White.png", x = (spawnx + randx), y = (spawny), v_y = vy, duration = randDuration, width = 8, height = 8)]
        for p in self.particles:
            self.add_widget(p)
            
########################################################
########################################################
########################################################
##                                                    ##
##                       MENUS                        ##
##                                                    ##
########################################################
########################################################
########################################################
            
class ScreenManagement(ScreenManager):
    #function to start the game from the title screen.
    def playGame(self):
        #Create a new game screen and add it to the screen manager
        gameScreen = GameScreen()
        self.add_widget(gameScreen)
               
        #Code from Assignment 3 to retrieve menu settings before entering the game.
        #Not currently used but may be useful in the future
        #titleScreen = self.get_screen('Title')
        #colors = {'p1':titleScreen.ids.p1color.color, 'p2':titleScreen.ids.p2color.color}
               
        #Initialize the game and set the current screen to the game screen.
        gameScreen.startGame()
        self.current = 'GameScreen'
    
    #Go to the titleScreen from the GameOver screen.
    def titleScreen(self):
        self.remove_widget(self.get_screen('GameOver'))
        self.current = 'Title'
        
    #Go to the game over screen
    def gameOver(self, text):
        screen = GameOver()
        screen.setText(text)
        self.add_widget(screen)
        self.remove_widget(self.current_screen)
        self.current = 'GameOver'
        
#Game screen class
class GameScreen(Screen):  
    #Start game function which creates an instance of the game when the game is ready to be played by the user.
    def startGame(self):
        game = Game()
        self.add_widget(game)
        
#Title screen class.
class TitleScreen(Screen):
    pass

#Game over screen
class GameOver(Screen):    
    #Set the text on the screen to the player who won the game.
    #Not currently used.
    def setText(self, text):
        self.ids.winText.text = text
        
#Global screen manager (because it wouldn't work any other way)
sm = ScreenManagement()
sm.add_widget(TitleScreen())

########################################################
########################################################
########################################################
##                                                    ##
##                    App Init                        ##
##                                                    ##
########################################################
########################################################
########################################################
                 
class GameApp(App):
    def build(self):
        sm.current = 'Title'
        return sm
        
GameApp().run()

[INFO   ] [Logger      ] Record log in C:\Users\Quinn\.kivy\logs\kivy_18-03-26_4.txt
[INFO   ] [Kivy        ] v1.10.0
[INFO   ] [Python      ] v3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
[INFO   ] [Factory     ] 194 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil, img_gif (img_ffpyplayer ignored)
[INFO   ] [Text        ] Provider: sdl2
[INFO   ] [OSC         ] using <thread> for socket
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GLEW initialization succeeded
[INFO   ] [GL          ] Backend used <glew>
[INFO   ] [GL          ] OpenGL version <b'4.5.0 - Build 22.20.16.4749'>
[INFO   ] [GL          ] OpenGL vendor <b'Intel'>
[INFO   ] [GL          ] OpenGL renderer <b'Intel(R) HD Graphics 620'>
[INFO   ] [GL          ] OpenGL parsed version: 4, 5
[INFO   ] [GL          ] Shading version <b'4.50 - Build 22.20.16.4749'>
[